In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train.tail()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
6250,6251,45,31/08/2012,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,False,734297.87
6251,6252,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66
6252,6253,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27
6253,6254,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20
6254,6255,45,28/09/2012,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,8.684,False,713173.95


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            6255 non-null   int64  
 1   Store         6255 non-null   int64  
 2   Date          6255 non-null   object 
 3   Temperature   6255 non-null   float64
 4   Fuel_Price    6255 non-null   float64
 5   Promotion1    2102 non-null   float64
 6   Promotion2    1592 non-null   float64
 7   Promotion3    1885 non-null   float64
 8   Promotion4    1819 non-null   float64
 9   Promotion5    2115 non-null   float64
 10  Unemployment  6255 non-null   float64
 11  IsHoliday     6255 non-null   bool   
 12  Weekly_Sales  6255 non-null   float64
dtypes: bool(1), float64(9), int64(2), object(1)
memory usage: 592.6+ KB


In [5]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i,col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col +=1
            print('결측치가 있는 칼럼은: %s입니다.'%col)
            print('해당 칼럼에 총 %s개의 결측치가 존재합니다. '%missing_values)
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다.')
    return missing_col

missing_col = check_missing_col(train)

결측치가 있는 칼럼은: Promotion1입니다.
해당 칼럼에 총 4153개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: Promotion2입니다.
해당 칼럼에 총 4663개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: Promotion3입니다.
해당 칼럼에 총 4370개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: Promotion4입니다.
해당 칼럼에 총 4436개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: Promotion5입니다.
해당 칼럼에 총 4140개의 결측치가 존재합니다. 


In [6]:
# 결측치 0으로 채워주고 로그변환을 위해 음수값들 0으로 바꿔주기
# Promotion 값들이 극단값이 많으므로 로그변환 해주기

# Promotion_list = ['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']

# for i in Promotion_list:
#     train[i].fillna(0,inplace=True)
#     train[i][train[i] < 0] = 0
#     train[i] = np.log1p(train[i])
    
# for i in Promotion_list:
#     test[i].fillna(0,inplace=True)
#     test[i][test[i] < 0] = 0
#     test[i] = np.log1p(test[i])


## Date를 년/월/일로 나눠주기

In [7]:
def get_month(date):
    month = date[3:5]
    month = int(month)
    return month

def get_day(date):
    day = date[:2]
    day = int(day)
    return day

def get_year(date):
    year = date[6:]
    year = int(year)
    return year



train['Month'] = train['Date'].apply(get_month)
train['Year'] = train['Date'].apply(get_year)
train['Day'] = train['Date'].apply(get_day)

test['Month'] = test['Date'].apply(get_month)
test['Year'] = test['Date'].apply(get_year)
test['Day'] = test['Date'].apply(get_day)

train

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Month,Year,Day
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90,2,2010,5
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44,2,2010,12
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17,2,2010,19
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59,2,2010,26
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68,3,2010,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251,45,31/08/2012,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,False,734297.87,8,2012,31
6251,6252,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66,9,2012,7
6252,6253,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27,9,2012,14
6253,6254,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20,9,2012,21


In [8]:
# def week(df):
#     df.Date = pd.to_datetime(df.Date, format="%d/%m/%Y")
#     df['Week'] = df.Date.dt.isocalendar().week.apply(lambda x: int(x))
#     return df

# train = week(train)
# test = week(test)

## test data를 확인해보면 2012.10을 예측하는 것임을 알 수 있음 
### -> 그렇다면 2010.11월과 12월, 2011.11월과 12월의 정보가 과연 필요할까? 라는 생각으로 삭제.

In [9]:
train = train[train['Month']<=10]
train

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Month,Year,Day
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90,2,2010,5
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44,2,2010,12
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17,2,2010,19
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59,2,2010,26
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68,3,2010,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,6251,45,31/08/2012,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,False,734297.87,8,2012,31
6251,6252,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66,9,2012,7
6252,6253,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27,9,2012,14
6253,6254,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20,9,2012,21


In [10]:
def holiday_to_number(isholiday):
    if isholiday == True:
        number = 1
    else:
        number = 0
    return number

In [11]:
train['Holiday'] = train['IsHoliday'].apply(holiday_to_number)
test['Holiday'] = test['IsHoliday'].apply(holiday_to_number)

In [12]:
train.drop(['id'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)

In [13]:
train

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Month,Year,Day,Holiday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90,2,2010,5,0
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44,2,2010,12,1
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17,2,2010,19,0
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59,2,2010,26,0
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68,3,2010,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,45,31/08/2012,75.09,3.867,23641.30,6.00,92.93,6988.31,3992.13,8.684,False,734297.87,8,2012,31,0
6251,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66,9,2012,7,1
6252,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27,9,2012,14,0
6253,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20,9,2012,21,0


In [22]:
from catboost import CatBoostRegressor


models = []

features = ['Store','Holiday','Month','Year','Day']


# 학습을 위한 하이퍼파라미터 설정
# 파라미터 설정을 안해주는게 성능이 더 높게 나오는듯?
model_params = {
    "n_estimators": 3000,
    "max_depth": 3,
   "learning_rate" : 0.01,
}

for store in range(1,max(train.Store)+1):
    train_store = train[train.Store==store]
    model = CatBoostRegressor(**model_params)
  #  model = CatBoostRegressor()
    model.fit(train_store[features], train_store.Weekly_Sales)
    models.append(model)

# pred_train = []
# for store in range(1, max(train.Store)+1):
#     train_store = train[train.Store==store]
#     y_pred = models[store-1].predict(train_store[features])
#     pred_train += y_pred.tolist()
    
pred = []
for store in range(1, max(train.Store)+1):
    test_store = test[test.Store==store]
    
    y_pred = models[store-1].predict(test_store[features])
    pred += y_pred.tolist()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1000:	learn: 6364.6615335	total: 359ms	remaining: 716ms
1001:	learn: 6362.2887717	total: 359ms	remaining: 715ms
1002:	learn: 6360.5002236	total: 359ms	remaining: 714ms
1003:	learn: 6359.3766191	total: 359ms	remaining: 713ms
1004:	learn: 6356.7651293	total: 359ms	remaining: 712ms
1005:	learn: 6355.6123743	total: 359ms	remaining: 711ms
1006:	learn: 6353.1355969	total: 359ms	remaining: 711ms
1007:	learn: 6351.1781683	total: 359ms	remaining: 710ms
1008:	learn: 6349.4158597	total: 359ms	remaining: 709ms
1009:	learn: 6348.7236841	total: 359ms	remaining: 708ms
1010:	learn: 6346.6122359	total: 359ms	remaining: 707ms
1011:	learn: 6344.2562186	total: 359ms	remaining: 706ms
1012:	learn: 6341.8468159	total: 359ms	remaining: 705ms
1013:	learn: 6340.3181492	total: 360ms	remaining: 704ms
1014:	learn: 6339.0024774	total: 360ms	remaining: 703ms
1015:	learn: 6337.5830200	total: 360ms	remaining: 702ms
1016:	learn: 6336.6978897	total: 360ms	remaining: 701ms
1017:	learn:

In [23]:
submission['Weekly_Sales'] = pred
submission.to_csv('submission_vacation_0804_3.csv', index=False)
submission

# submission['Weekly_Sales'] = pred_sub
# submission.to_csv('baseline_vacation_0722_1.csv', index=False)

,id,Weekly_Sales
0,1,1.723231e+06
1,2,1.581016e+06
2,3,1.556604e+06
3,4,1.487566e+06
4,5,1.920742e+06
...,...,...
175,176,3.544832e+05
176,177,7.689516e+05
177,178,7.335435e+05
178,179,7.373300e+05


In [ ]:
from sklearn.model_selection import cross_val_score

# 앞에서 모델 구축한 후 진행 
# train_test_split하지 않고 train set 전체 사용

# xgb_reg.fit(x, y)


neg_mse_scores = cross_val_score(cat, X_train, y_train, scoring="neg_mean_squared_error", cv = 5)
rmse_scores  = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

# cross_val_score(scoring="neg_mean_squared_error")로 반환된 값은 모두 음수 
# print(' 5 folds 의 개별 Negative MSE scores: ', np.round(neg_mse_scores, 2))
print(' 5 folds 의 개별 RMSE scores : ', np.round(rmse_scores, 2))
print(' 5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))

# 5 folds 의 평균 RMSE : 73975.857 

# 5 folds 의 평균 RMSE : 72175.749 

# 5 folds 의 평균 RMSE : 79026.304 